In [1]:
import json
import pandas as pd

# Load the JSON file
with open("dev_tables.json", "r", encoding="utf-8") as f:
    data = json.load(f)

table_schemas = {}
dbs = []

for db in data:
    db_id = db.get("db_id", [])
    dbs.append(db_id)
    schema_promt = f"Below are the descriptions for the database {db_id}:\n"
    for table in db.get("table_names_original", []):
        table_index = db["table_names_original"].index(table)
        schema_promt += f"[Table name]\n{table}\n[(Column name, type)]\n"
        for i, (tbl_idx, col_name) in enumerate(db.get("column_names_original", [])):
            if tbl_idx == table_index:
                type_desc = db.get("column_types", [])[i]
                schema_promt += f"{col_name}, {type_desc}\n"
        schema_promt += "\n"
    table_schemas[db_id] = schema_promt

In [2]:
from openai import OpenAI

results = {}
for db_id in dbs:
    results[db_id] = []

client = OpenAI()

with open(f"mini_dev_sqlite.json", "r", encoding="utf-8") as f:
    queries = json.load(f)

for query in queries:
    #generate full prompt for queries
    db_id = query["db_id"]
    prompt = table_schemas[db_id]
    prompt += f"Given the above database schema, generate SQL query for the below question:\n"
    prompt += f"{query['question']}\nReturn a single SQL with no explanation.\nBegin!"
    response = ""
    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                "role": "user",
                "content": prompt
                }
            ],
            temperature=0
        )
        response = completion.choices[0].message.content
    except Exception as e:
        pass
    # print(query["question"])
    # print(query["query"])
    # print(response)
    results[db_id].append({
        "question": query["question"],
        "gold_query": query["SQL"],
        "generated_query": response
    })

In [3]:
#save results to file
with open("results_schema_only.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2)